In [ ]:
import sys
sys.path.append('../../')

In [ ]:
import sys
sys.path.append('../../')
from SynTemp.SynAAM.aam_postprocess import AMMPostprocessor
from SynTemp.SynITS.its_extraction import ITSExtraction
from SynTemp.SynUtils.utils import load_database, save_database

data = load_database(f'../../Data/uspto/uspto_balance_aam_reactions.json.gz')
mapper_name = ['rxn_mapper', 'graphormer', 'local_mapper']

# make sure reactions all have atom mapping
check_valid = AMMPostprocessor.parallel_postprocess(data, mapper_name, threshold=3, n_jobs=5, verbose=1)
valid_data = [reaction for reaction in check_valid if reaction.get('Valid')]

In [ ]:
# Extract rules and graph rules based on Imaginary Transition State (ITS)
its_graph, its_graph_wrong = ITSExtraction.parallel_process_smiles(valid_data, mapper_name, threshold=2, n_jobs=5, verbose=1, export_full=False, check_method='RC')

len(its_graph)